### Globale Funktionen

In [1]:
def euclidean(loc1, loc2):
    return ((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)**0.5

### Klasse Request

In [9]:
class Request:
    
    def __init__(self, orig_loc, dest_loc, req_t, alpha):
        self.orig_loc = orig_loc
        self.dest_loc = dest_loc
        self.req_t = req_t
        self.alpha = alpha # tolerierte Wartezeit
        self.end_t = req_t + euclidean(orig_loc, dest_loc) + alpha
    
    orig_earl = -1
    dest_earl = -1
    orig_late = -1
    dest_late = -1

### Klasse Tour

In [17]:
class Tour:
    
    def __init__(self, depot_loc, M=10000):
        depot = Request(depot_loc, depot_loc, 0, M)
        depot.orig_earl = 0
        depot.dest_earl = 0
        depot.orig_late = M
        depot.dest_late = M
        self.tour = [(depot, "orig"), (depot, "dest")]
        self.length = 2
    
    def get_coordinates(self, tup):
        return tup[0].orig_loc if tup[1] == "orig" else tup[0].dest_loc
    
    def get_earliest(self, tup):
        return tup[0].orig_earl if tup[1] == "orig" else tup[0].dest_earl
    
    def get_latest(self, tup):
        return tup[0].orig_late if tup[1] == "orig" else tup[0].dest_late
    
    def get_insert_earl_late(self, ins, bef, aft):
        earliest = max(ins[0].req_t, 
                       self.get_earliest(bef) + euclidean(self.get_coordinates(bef), self.get_coordinates(ins)))
        latest = min(ins[0].end_t,
                    self.get_latest(aft) - euclidean(self.get_coordinates(ins) - self.get_coordinates(aft)))
        return earliest, latest
    
    def update_earliest(self, tupA, tupB):
        locA = self.get_coordinates(tupA)
        locB = self.get_coordinates(tupB)
        earlA = self.get_earliest(tupA)
        earlB = self.get_earliest(tupB)
        new = max(earlA, earlB + euclidan(locA, locB))
        (tupA.orig_earl = new) if tupA[1] == "orig" else (tupA.dest_earl = new)
        
    def update_latest(self, tupA, tupB):
        locA = self.get_coordinates(tupA)
        locB = self.get_coordinates(tupB)
        lateA = self.get_latest(tupA)
        lateB = self.get_latest(tupB)
        new = min(lateA, lateB - euclidean(locA, locB))
        (tupA.orig_late = new) if tupA[1] == "orig" else (tupA.dest_late = new)
    
    def feasible(self, i, place):
        pass
    
    def insertion_cost(self, i, place):
        bef = self.get_coordinates(self.tour[i-1])
        ins = self.get_coordinates(place)
        aft = self.get_coordinates(self.tour[i])
        return euclidean(bef, ins) + euclidean(ins, aft) - euclidean(bef, aft)
    
    def insert(self, i, place):
        self.tour.insert(i, place)
        self.length += 1
        self.update(i)
    
    def update(self, i):
        earliest, latest = self.get_insert_earl_late():
        if self.tour[i][1] == "orig":
            self.tour[i][0].orig_earl = earliest
            self.tour[i][0].orig_late = latest
        else:
            self.tour[i][0].dest_earl = earliest
            self.tour[i][0].dest_late = latest
        
        for k in range(i-1, -1, -1):
            self.update_latest(self.tour[k], self.tour[k-1])
        
        for k in range(i, self.length+1):
            self.update_earliest(self.tour[k], self.tour[k+1])
            

### Parallel Insertion

In [16]:
def parallel_insertion(tours, places, M=10000):
    while places:
        cOpt = M
        for place in places:
            for tour in tours:
                for i in range(1, tour.length):
                    if tour.feasible(i, place) and tour.insertion_cost(i, place) < cOpt:
                        tourOpt = tour
                        iOpt = i
                        placeOpt = place
                        cOpt = tour.insertion:cost(i, place)
        tourOpt.insert(iOpt, placeOpt)
        places.remove(placeOpt)
        tourOpt.update(iOpt)

### Test